In [1]:

import os
import copy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from typing import Optional, Callable
from sklearn.metrics import accuracy_score, f1_score
from sklearn.covariance import LedoitWolf
import json

from torch.utils.tensorboard import SummaryWriter # For Tensorboard
from collections.abc import Mapping

from torchvision.transforms import v2

torch.manual_seed(0)
%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
print(device)

cpu


/home/bluedrack/.conda/envs/iapr/lib/python3.9/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
class CoinDataset(Dataset):
    """DHMC dataset using 2 classes"""

    def __init__(self, features_path : str, data_json, transform : bool = False) -> None:
        """
        Attributes:
            raw_data (list of dict): (M) List of M slides raw data as dictionaries. 
            train (bool): True if data are the training set. False otherwise
            
        Args:
            features_path (str): The path to the features file
            train (bool): Whether it is the training dataset or not
        """
        
        super().__init__()

        self.raw_data = []
        # data_json = None
        # with open(label_path, 'r') as f:
        #     data_json = json.load(f)
        for x in data_json:
            filename = x["filename"]
            img_path = f"{features_path}/{filename}"
            img = Image.open(img_path)
            self.raw_data.append({"image_features": img, "label": int(x["value"])})
        
        if transform:
            self.transform = v2.Compose([
                v2.Resize(224),
                v2.CenterCrop(224),
                v2.RandomRotation(degrees=(0, 300)),
                v2.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.4, hue=0.3),
                v2.ToTensor(),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                v2.RandomErasing(p=0.5)
                ])
        else:
            self.transform = v2.Compose([
                v2.Resize(224),
                v2.CenterCrop(224),
                v2.ToTensor(),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

    def __len__(self) -> int:
        """Returns the length of the dataset

        Returns:
            int: The length M of the dataset
        """

        n_data = len(self.raw_data)
        return n_data
    
    def __getitem__(self, index : int):
        """Returns the entry at index from the dataset

        Args:
            index (int): the requested entry index of the dataset

        Returns:
            features (torch.Tensor): (N, d) Feature tensor of the selected slide with N patches and d feature dimensions
            label (int): Ground truth label {0, ..., n_classes}
            wsi_id (str): Name of the WSI as "DHMC_xxx" where xxx is a unique id of the slide (train == False only)
            coordinates (torch.Tensor): (N, 2) xy coordinates of the N patches of the selected slide (train == False only)
        """

        features = None
        label = None

        features = self.raw_data[index]["image_features"]
        label = torch.tensor(self.raw_data[index]["label"])
    
        features = self.transform(features)
        return features, label


In [4]:
# Load all the img from output folder and make a dataset
import random
from sklearn.model_selection import train_test_split

data_json = None
with open('data-fixed.json', 'r') as f:
    data_json = json.load(f)

# final_data = []
# for x in data_json:
#     final_data.append({"filename": x["filename"], "value": x["value"]})
train_data, test_data = train_test_split(data_json, test_size=0.2, random_state=42)
train_dataset = CoinDataset('./output/', train_data, transform=True)
test_dataset = CoinDataset('./output/', test_data, transform=False)

# train_size = int(0.8 * len(coinDataset))
# test_size = len(coinDataset) - train_size

# train_dataset, test_dataset = torch.utils.data.random_split(coinDataset, [train_size, test_size])
# val_coinDataset = CoinDataset('./output/', "val_data_split.json", transform=False)
# trans = tranosforms.Compose([transforms.Resize(256),transforms.ToTensor()])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
# dataset = datasets.ImaVgeFolder('./output-1/', transform=transforms.ToTensor())

/home/bluedrack/.conda/envs/iapr/lib/python3.9/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [5]:
# from torchvision.models import resnet50, ResNet50_Weights

# model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
# model.fc = nn.Sequential(
#     nn.Dropout(0.5),
#     nn.Linear(2048, 256),
#     nn.BatchNorm1d(256),
#     nn.ReLU(),
#     nn.Linear(256, 16)
# )

# from torchvision.models import resnet18, ResNet18_Weights

# model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
# model.fc = nn.Sequential(
#     nn.Dropout(0.5),
#     nn.Linear(512, 256),
#     nn.BatchNorm1d(256),
#     nn.ReLU(),
#     nn.Linear(256, 16)
# )

from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights



model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    # nn.Linear(1280, 256),
    # nn.BatchNorm1d(256),
    # nn.ReLU(),
    nn.Linear(1280, 16)
)
# model.eval()
# from model import ResNetModel
# model = ResNetModel()



In [6]:

def train(model, train_loader, val_loader, optimizer, scheduler, criterion, epochs=10):
    
    model.to(device)
    steps = 0
    best_epoch = 0
    for epoch in range(epochs):
        model.train()
        for i, (data, target) in enumerate(train_loader):
        
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            
            loss.backward()
            optimizer.step()
            acc = accuracy_score(target, output.argmax(dim=1, keepdim=True))
            writer.add_scalar("Acc/train", acc, steps)
            writer.add_scalar("Loss/train", loss, steps)
            steps+=1
            if i % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Acc: {:.3f}'.format(
                    epoch, i * len(data), len(train_loader.dataset),
                    100. * i / len(train_loader), loss.item(), acc*100))
                

        running_vloss = []
        val_f1micro = []
        val_f1macro = []
        correct = 0
        model.eval()
        with torch.no_grad():
            for i, vdata in enumerate(val_loader):
                vinputs, vlabels = vdata
                voutputs = model(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss.append(vloss)
                pred = voutputs.argmax(dim=1, keepdim=True)
                correct += pred.eq(vlabels.view_as(pred)).sum().item()
                val_f1micro.append(f1_score(pred, vlabels, average='micro'))
                val_f1macro.append(f1_score(pred, vlabels, average='macro'))

        avg_vloss = np.mean(running_vloss)
        avg_vf1micro = np.mean(val_f1micro)
        avg_vf1macro = np.mean(val_f1macro)

        val_accuracy = 100. * correct / len(val_loader.dataset)
        print('Val Epoch: {}\tLoss: {:.6f}, Acc: {:.3f}, F1 micro: {:.3f}'.format(
            epoch, avg_vloss, val_accuracy, avg_vf1micro
        ))

        writer.add_scalar("Loss/val", avg_vloss, (epoch + 1))
        writer.add_scalar("Acc/val", val_accuracy, (epoch + 1))
        writer.add_scalar("F1_micro/val", avg_vf1micro, epoch + 1)
        writer.add_scalar("F1_macro/val", avg_vf1macro, epoch + 1)
        scheduler.step(avg_vloss)
        if best_epoch < val_accuracy:
            best_epoch = val_accuracy
            torch.save(model, "model/best_model.pth")
                
# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Define the scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# Log in Tensorboard
writer = SummaryWriter('runs/efficientnet_v2_s_baseline')

train(model, train_loader, val_loader, optimizer, scheduler, nn.CrossEntropyLoss(), epochs=35)
writer.flush()
writer.close()

# Save the model
torch.save(model, "model/model.pth")

Train Epoch: 0 [0/305 (0%)]	Loss: 2.807038, Acc: 6.250
Val Epoch: 0	Loss: 2.665947, Acc: 18.182, F1 micro: 0.182
Train Epoch: 1 [0/305 (0%)]	Loss: 2.768898, Acc: 6.250
Val Epoch: 1	Loss: 2.526230, Acc: 44.156, F1 micro: 0.442
Train Epoch: 2 [0/305 (0%)]	Loss: 2.637392, Acc: 9.375
Val Epoch: 2	Loss: 2.344788, Acc: 45.455, F1 micro: 0.455
Train Epoch: 3 [0/305 (0%)]	Loss: 2.453512, Acc: 34.375
Val Epoch: 3	Loss: 2.110254, Acc: 42.857, F1 micro: 0.429
Train Epoch: 4 [0/305 (0%)]	Loss: 2.208323, Acc: 40.625


In [60]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 78.829MB
